In [1]:
import pandas as pd
import numpy as np
import os
import math
import re
from collections import Counter
import random

import scipy.stats
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties  # 导入FontProperties
import warnings
warnings.filterwarnings("ignore")

In [2]:
import statsmodels.api as sm

In [3]:
path1 = r"E:\netData\APS\3 Paper\nodepro2010.zip"
path2 = r"E:\netData\APS\DCCC\paper DCCC2010.zip"

In [4]:
df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)

In [5]:
df = pd.concat([df1,df2],axis = 1)

In [6]:
df = df.fillna(0)
df['teamsize'] = df['team size']

In [7]:
df

,degree,indegree,outdegree,team size,cit5,year,journal,meta,closeness,betweenness,...,CD,2stepD,D,id,first_cit_y,TCRef,id,DC,CC,teamsize
0,1,1,0,1,1.0,1913,Physical Review,Physical Review1913,0.666667,0.000000e+00,...,1.000000,1.000000,1.000000,10.1103/PhysRev.1.1,0,0,10.1103/PhysRev.1.1,1,0,1
1,5,5,0,1,0.0,1913,Physical Review,Physical Review1913,0.159071,0.000000e+00,...,1.000000,1.000000,1.000000,10.1103/PhysRev.1.124,10,0,10.1103/PhysRev.1.124,5,0,1
2,0,0,0,1,0.0,1913,Physical Review,Physical Review1913,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,10.1103/PhysRev.1.141,0,0,10.1103/PhysRev.1.141,0,0,1
3,0,0,0,1,0.0,1913,Physical Review,Physical Review1913,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,10.1103/PhysRev.1.154,0,0,10.1103/PhysRev.1.154,0,0,1
4,4,4,0,1,3.0,1913,Physical Review,Physical Review1913,0.157664,0.000000e+00,...,1.000000,1.000000,1.000000,10.1103/PhysRev.1.16,1,0,10.1103/PhysRev.1.16,4,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463343,19,10,9,1,5.0,1937,Reviews of Modern Physics,Reviews of Modern Physics1937,0.199643,1.419043e+07,...,-0.047244,0.006154,0.015748,10.1103/RevModPhys.9.391,1,171,10.1103/RevModPhys.9.391,2,8,1
463344,2,0,2,1,0.0,1937,Reviews of Modern Physics,Reviews of Modern Physics1937,0.131657,0.000000e+00,...,0.000000,0.000000,0.000000,10.1103/RevModPhys.9.403,0,18,10.1103/RevModPhys.9.403,0,0,1
463345,56,50,6,1,1.0,1937,Reviews of Modern Physics,Reviews of Modern Physics1937,0.222096,1.784951e+06,...,0.159420,0.039454,0.170290,10.1103/RevModPhys.9.432,3,235,10.1103/RevModPhys.9.432,47,3,1
463346,24,4,20,1,2.0,1937,Reviews of Modern Physics,Reviews of Modern Physics1937,0.165863,3.392220e+06,...,-0.030769,0.013699,0.015385,10.1103/RevModPhys.9.44,0,121,10.1103/RevModPhys.9.44,1,3,1


In [8]:
df['journal'] = df['journal'].map(dict(zip(df['journal'].unique() , [0,1,2,3,4,5,6,7,0,0,8])))

In [9]:
scipy.stats.kendalltau(df['CD'] , df['outdegree'])

KendalltauResult(correlation=-0.16771310651944124, pvalue=0.0)

In [10]:
scipy.stats.kendalltau(df['DC'] , df['outdegree'])

KendalltauResult(correlation=-0.07571532947590792, pvalue=0.0)

In [12]:
scipy.stats.kendalltau(df['CD'] , df['TCRef'])

KendalltauResult(correlation=-0.08661338206937483, pvalue=0.0)

In [13]:
scipy.stats.kendalltau(df['DC'] , df['TCRef'])

KendalltauResult(correlation=-0.005658395902436871, pvalue=1.4966533684288247e-07)

In [11]:
scipy.stats.kendalltau(df['indegree'] , df['outdegree'])

KendalltauResult(correlation=0.14783366649385105, pvalue=0.0)

In [20]:
formula = 'CD ~  C(year) + C(journal) + outdegree + teamsize + first_cit_y'
# fit the OLS model
model = sm.formula.ols(formula=formula, data=df).fit()
print(model.summary())

params = model.params
bse = model.bse
pvalues = model.pvalues
# create a DataFrame with the results
results_df = pd.DataFrame({'coef': params,'std err': bse,'p_value': pvalues})
results_df = results_df.round(4)
# add asterisks to the p-values based on the significance level
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['p_value'] = results_df['p_value'].astype(float)
results_df['p_star'] = ''
results_df.loc[results_df['p_value'] < 0.01, 'p_star'] = '***'
results_df.loc[(results_df['p_value'] >= 0.01) & (results_df['p_value'] < 0.05), 'p_star'] = '**'
results_df.loc[(results_df['p_value'] >= 0.05) & (results_df['p_value'] < 0.1), 'p_star'] = '*'
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['v'] = results_df['coef'].astype(str) + results_df['p_star'] +'\n'+ '('+ results_df['std err'].astype(str) +')'
results_df.to_excel('E:/1.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     CD   R-squared:                       0.075
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     297.6
Date:                Tue, 18 Apr 2023   Prob (F-statistic):               0.00
Time:                        15:18:30   Log-Likelihood:             1.5855e+05
No. Observations:              463348   AIC:                        -3.168e+05
Df Residuals:                  463221   BIC:                        -3.154e+05
Df Model:                         126                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        3.253e+08   1.43e+08     

In [21]:
formula = 'CD ~  C(year) + C(journal) + outdegree + indegree + teamsize + first_cit_y'
# fit the OLS model
model = sm.formula.ols(formula=formula, data=df).fit()
print(model.summary())

params = model.params
bse = model.bse
pvalues = model.pvalues
# create a DataFrame with the results
results_df = pd.DataFrame({'coef': params,'std err': bse,'p_value': pvalues})
results_df = results_df.round(4)
# add asterisks to the p-values based on the significance level
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['p_value'] = results_df['p_value'].astype(float)
results_df['p_star'] = ''
results_df.loc[results_df['p_value'] < 0.01, 'p_star'] = '***'
results_df.loc[(results_df['p_value'] >= 0.01) & (results_df['p_value'] < 0.05), 'p_star'] = '**'
results_df.loc[(results_df['p_value'] >= 0.05) & (results_df['p_value'] < 0.1), 'p_star'] = '*'
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['v'] = results_df['coef'].astype(str) + results_df['p_star'] +'\n'+ '('+ results_df['std err'].astype(str) +')'
results_df.to_excel('E:/2.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     CD   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                  0.076
Method:                 Least Squares   F-statistic:                     301.1
Date:                Tue, 18 Apr 2023   Prob (F-statistic):               0.00
Time:                        15:18:36   Log-Likelihood:             1.5889e+05
No. Observations:              463348   AIC:                        -3.175e+05
Df Residuals:                  463220   BIC:                        -3.161e+05
Df Model:                         127                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        3.543e+08   1.43e+08     

In [22]:
formula = 'DC ~  C(year) + C(journal) + outdegree + teamsize + first_cit_y'
# fit the OLS model
model = sm.formula.ols(formula=formula, data=df).fit()
print(model.summary())

params = model.params
bse = model.bse
pvalues = model.pvalues
# create a DataFrame with the results
results_df = pd.DataFrame({'coef': params,'std err': bse,'p_value': pvalues})
results_df = results_df.round(4)
# add asterisks to the p-values based on the significance level
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['p_value'] = results_df['p_value'].astype(float)
results_df['p_star'] = ''
results_df.loc[results_df['p_value'] < 0.01, 'p_star'] = '***'
results_df.loc[(results_df['p_value'] >= 0.01) & (results_df['p_value'] < 0.05), 'p_star'] = '**'
results_df.loc[(results_df['p_value'] >= 0.05) & (results_df['p_value'] < 0.1), 'p_star'] = '*'
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['v'] = results_df['coef'].astype(str) + results_df['p_star'] +'\n'+ '('+ results_df['std err'].astype(str) +')'
results_df.to_excel('E:/5.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     DC   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     104.9
Date:                Tue, 18 Apr 2023   Prob (F-statistic):               0.00
Time:                        15:18:44   Log-Likelihood:            -2.0040e+06
No. Observations:              463348   AIC:                         4.008e+06
Df Residuals:                  463221   BIC:                         4.010e+06
Df Model:                         126                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        2.027e+09   1.52e+10     

In [23]:
formula = 'DC ~  C(year) + C(journal) + outdegree + indegree + teamsize + first_cit_y'
# fit the OLS model
model = sm.formula.ols(formula=formula, data=df).fit()
print(model.summary())

params = model.params
bse = model.bse
pvalues = model.pvalues
# create a DataFrame with the results
results_df = pd.DataFrame({'coef': params,'std err': bse,'p_value': pvalues})
results_df = results_df.round(4)
# add asterisks to the p-values based on the significance level
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['p_value'] = results_df['p_value'].astype(float)
results_df['p_star'] = ''
results_df.loc[results_df['p_value'] < 0.01, 'p_star'] = '***'
results_df.loc[(results_df['p_value'] >= 0.01) & (results_df['p_value'] < 0.05), 'p_star'] = '**'
results_df.loc[(results_df['p_value'] >= 0.05) & (results_df['p_value'] < 0.1), 'p_star'] = '*'
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['v'] = results_df['coef'].astype(str) + results_df['p_star'] +'\n'+ '('+ results_df['std err'].astype(str) +')'
results_df.to_excel('E:/6.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     DC   R-squared:                       0.766
Model:                            OLS   Adj. R-squared:                  0.766
Method:                 Least Squares   F-statistic:                 1.193e+04
Date:                Tue, 18 Apr 2023   Prob (F-statistic):               0.00
Time:                        15:18:50   Log-Likelihood:            -1.6741e+06
No. Observations:              463348   AIC:                         3.349e+06
Df Residuals:                  463220   BIC:                         3.350e+06
Df Model:                         127                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         9.03e+08   7.46e+09     

In [25]:
formula = 'CD ~  C(year) + C(journal)  + TCRef + teamsize + first_cit_y'
# fit the OLS model
model = sm.formula.ols(formula=formula, data=df).fit()
print(model.summary())

params = model.params
bse = model.bse
pvalues = model.pvalues
# create a DataFrame with the results
results_df = pd.DataFrame({'coef': params,'std err': bse,'p_value': pvalues})
results_df = results_df.round(4)
# add asterisks to the p-values based on the significance level
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['p_value'] = results_df['p_value'].astype(float)
results_df['p_star'] = ''
results_df.loc[results_df['p_value'] < 0.01, 'p_star'] = '***'
results_df.loc[(results_df['p_value'] >= 0.01) & (results_df['p_value'] < 0.05), 'p_star'] = '**'
results_df.loc[(results_df['p_value'] >= 0.05) & (results_df['p_value'] < 0.1), 'p_star'] = '*'
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['v'] = results_df['coef'].astype(str) + results_df['p_star'] +'\n'+ '('+ results_df['std err'].astype(str) +')'
results_df.to_excel('E:/3.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     CD   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     221.3
Date:                Tue, 18 Apr 2023   Prob (F-statistic):               0.00
Time:                        15:23:50   Log-Likelihood:             1.5406e+05
No. Observations:              463348   AIC:                        -3.079e+05
Df Residuals:                  463221   BIC:                        -3.065e+05
Df Model:                         126                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.0804      0.001     

In [26]:
formula = 'CD ~  C(year) + C(journal)  + TCRef + indegree + teamsize + first_cit_y'
# fit the OLS model
model = sm.formula.ols(formula=formula, data=df).fit()
print(model.summary())

params = model.params
bse = model.bse
pvalues = model.pvalues
# create a DataFrame with the results
results_df = pd.DataFrame({'coef': params,'std err': bse,'p_value': pvalues})
results_df = results_df.round(4)
# add asterisks to the p-values based on the significance level
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['p_value'] = results_df['p_value'].astype(float)
results_df['p_star'] = ''
results_df.loc[results_df['p_value'] < 0.01, 'p_star'] = '***'
results_df.loc[(results_df['p_value'] >= 0.01) & (results_df['p_value'] < 0.05), 'p_star'] = '**'
results_df.loc[(results_df['p_value'] >= 0.05) & (results_df['p_value'] < 0.1), 'p_star'] = '*'
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['v'] = results_df['coef'].astype(str) + results_df['p_star'] +'\n'+ '('+ results_df['std err'].astype(str) +')'
results_df.to_excel('E:/4.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     CD   R-squared:                       0.057
Model:                            OLS   Adj. R-squared:                  0.057
Method:                 Least Squares   F-statistic:                     221.0
Date:                Tue, 18 Apr 2023   Prob (F-statistic):               0.00
Time:                        15:23:56   Log-Likelihood:             1.5415e+05
No. Observations:              463348   AIC:                        -3.080e+05
Df Residuals:                  463220   BIC:                        -3.066e+05
Df Model:                         127                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.0789      0.001     

In [27]:
formula = 'DC ~  C(year) + C(journal)  + TCRef + teamsize + first_cit_y'
# fit the OLS model
model = sm.formula.ols(formula=formula, data=df).fit()
print(model.summary())

params = model.params
bse = model.bse
pvalues = model.pvalues
# create a DataFrame with the results
results_df = pd.DataFrame({'coef': params,'std err': bse,'p_value': pvalues})
results_df = results_df.round(4)
# add asterisks to the p-values based on the significance level
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['p_value'] = results_df['p_value'].astype(float)
results_df['p_star'] = ''
results_df.loc[results_df['p_value'] < 0.01, 'p_star'] = '***'
results_df.loc[(results_df['p_value'] >= 0.01) & (results_df['p_value'] < 0.05), 'p_star'] = '**'
results_df.loc[(results_df['p_value'] >= 0.05) & (results_df['p_value'] < 0.1), 'p_star'] = '*'
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['v'] = results_df['coef'].astype(str) + results_df['p_star'] +'\n'+ '('+ results_df['std err'].astype(str) +')'
results_df.to_excel('E:/7.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     DC   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     105.3
Date:                Tue, 18 Apr 2023   Prob (F-statistic):               0.00
Time:                        15:24:04   Log-Likelihood:            -2.0040e+06
No. Observations:              463348   AIC:                         4.008e+06
Df Residuals:                  463221   BIC:                         4.010e+06
Df Model:                         126                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           6.1374      0.133     

In [28]:
formula = 'DC ~  C(year) + C(journal)  + TCRef + indegree + teamsize + first_cit_y'
# fit the OLS model
model = sm.formula.ols(formula=formula, data=df).fit()
print(model.summary())

params = model.params
bse = model.bse
pvalues = model.pvalues
# create a DataFrame with the results
results_df = pd.DataFrame({'coef': params,'std err': bse,'p_value': pvalues})
results_df = results_df.round(4)
# add asterisks to the p-values based on the significance level
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['p_value'] = results_df['p_value'].astype(float)
results_df['p_star'] = ''
results_df.loc[results_df['p_value'] < 0.01, 'p_star'] = '***'
results_df.loc[(results_df['p_value'] >= 0.01) & (results_df['p_value'] < 0.05), 'p_star'] = '**'
results_df.loc[(results_df['p_value'] >= 0.05) & (results_df['p_value'] < 0.1), 'p_star'] = '*'
results_df['p_value'] = results_df['p_value'].apply(lambda x: '{:.3f}'.format(x))
results_df['v'] = results_df['coef'].astype(str) + results_df['p_star'] +'\n'+ '('+ results_df['std err'].astype(str) +')'
results_df.to_excel('E:/8.xlsx')

                            OLS Regression Results                            
Dep. Variable:                     DC   R-squared:                       0.760
Model:                            OLS   Adj. R-squared:                  0.760
Method:                 Least Squares   F-statistic:                 1.158e+04
Date:                Tue, 18 Apr 2023   Prob (F-statistic):               0.00
Time:                        15:24:10   Log-Likelihood:            -1.6795e+06
No. Observations:              463348   AIC:                         3.359e+06
Df Residuals:                  463220   BIC:                         3.361e+06
Df Model:                         127                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -1.0144      0.066    -